# A3 Draw random realizations of galaxy clusters


This notebook will describe how to perform random draws of galaxy clusters from the statistical model trained in notebook A2 of the tutorial


## Objectives
    
    2 Draw random realizations from the KDE galaxy cluster model
    
    
## Setup

This notebook relies on the:


    * galaxy clusters member model from the previous calculation step: the output of the rejection sampling
    
    * synthetic software package. 


## Output

    *  synthetic galaxy cluster realizations
 
    
## Contact

In case of questions, contact me at t.varga@physik.lmu.de

In [1]:
from importlib import reload
import fitsio as fio
import numpy as np
import pandas as pd

import healpy as hp
import copy
import sys
import glob
import os
import matplotlib.pyplot as plt
%matplotlib inline

import sklearn.decomposition as decomp


import matplotlib as mpl
import subprocess as sp
import scipy.interpolate as interpolate
import pickle as pickle

import multiprocessing as mp

import synthetic.tools as tools
import synthetic.emulator.emulator as emulator
import synthetic.emulator.indexer as indexer
import synthetic.emulator.reader as reader


# Preparation (much the same as A2)

## Loading the regular data tables

First load the two primary obsered datasets, since this is a simulated scenarion both are comprehensive and contain all relevant quantities 

In [4]:
# CHANGE THE ROOT PATH TO YOUR OWN PATH
root_path = "/e/ocean1/users/vargatn/LSST/DC2_1.1.4/clusters_v01/"
deep_data_path = root_path + "dc2_cluster_sim_cutouts/cosmoDC2_v1.1.4_refpixels.h5"
wide_data_path = root_path + "dc2_cluster_sim_cutouts/clust_dc2-sim-LOS_v1.h5"

In [3]:
deep_smc_settings = {
    "columns": [
        ("GABS", ("ellipticity_1_true", "ellipticity_2_true", "SQSUM")),
        ("SIZE", "size_true"),
        ("MAG_I", "mag_i"),
        ("COLOR_G_R", ("mag_g", "mag_r", "-")),
        ("COLOR_R_I", ("mag_r", "mag_i", "-")),
        ("COLOR_I_Z", ("mag_i", "mag_z", "-")),
        ("STELLAR_MASS", "stellar_mass"),
        ("HALO_MASS", "halo_mass")        
    ],
    "logs": [False, True, False, False, False, False, True, True],
    "limits": [(0., 1.), (-1, 5), (17, 25), (-1, 3), (-1, 3), (-1, 3), (10**3, 10**13), (10**9, 10**16)],
}

In [5]:
wide_cr_settings = {
    "columns": [
        ("GABS", ("ellipticity_1_true", "ellipticity_2_true", "SQSUM")),
        ("SIZE", "size_true"),
        ("MAG_I", "mag_i"),
        ("COLOR_G_R", ("mag_g", "mag_r", "-")),
        ("COLOR_R_I", ("mag_r", "mag_i", "-")),
        ("COLOR_I_Z", ("mag_i", "mag_z", "-")),
        ("STELLAR_MASS", "stellar_mass"),
        ("HALO_MASS", "halo_mass"),
        ("LOGR", "R"),        
    ],
    "logs": [False, True, False, False, False, False, True, True, True],
    "limits": [(0., 1.), (-1, 5), (17, 25), (-1, 3), (-1, 3), (-1, 3), (10**3, 10**13), (10**9, 10**16), (1e-3, 16)],
}

In [7]:
refpixel = pd.read_hdf(deep_data_path, key="data")
table = pd.read_hdf(wide_data_path, key="data")

In [8]:

tmp_wide_cr_settings = wide_cr_settings.copy()
# tmp_wide_cr_settings["limits"][-1] = (10**-3, 10**LOGR_CAT_RMAXS[i])
_wide_cr_settings_rands = emulator.construct_deep_container(refpixel, tmp_wide_cr_settings)

# loading deep catalogs
_deep_smc_settings = emulator.construct_deep_container(refpixel, deep_smc_settings)

# loading cluster data
tmp_wide_cr_settings = wide_cr_settings.copy()
# tmp_wide_cr_settings["limits"][-1] = (10**-3, 10**LOGR_CAT_RMAXS[i])
_wide_cr_settings_clust = emulator.construct_deep_container(table, tmp_wide_cr_settings)

(1513572, 9)
(1513572,)
(1513572, 8)
(1513572,)
(1009658, 9)
(1009658,)


In [9]:
samples = []
scores = []
for rbin in np.arange(4):
    print(rbin)
#     expr = "/e/ocean1/users/vargatn/EMULATOR/EPSILON/resamples/epsilon_concentric_sample_v06_run*_rbin" + str(rbin) + "*samples.fits" 
    expr = "/e/ocean1/users/vargatn/LSST/DC2_1.1.4/clusters_v01/resamples/dc2-alpha_concentric_sample-v01_test-03/dc2-alpha_concentric_sample-v01_test-03_run0*_rbin" + str(rbin) + "*samples.fits" 

    fnames_samples = np.sort(glob.glob(expr))
    fnames_scores = []
    for fname in fnames_samples:
        fnames_scores.append(fname.replace("samples.fits", "scores.fits"))

    samples_sep = []
    scores_sep = []
    for i, fname in enumerate(fnames_samples):
#         print(fname)
        samples_sep.append(fio.read(fname))
        scores_sep.append(fio.read(fnames_scores[i]))
        
    samples.append(np.hstack(samples_sep))
    scores.append(np.hstack(scores_sep))

0
1
2
3


In [15]:
mag_lims = (17, 22.5)
r_lims_all = [(-1.5, -0.5), (-0.5, 0.), (0, 0.5), (0.5, 1.0)]
redges = [-1.5, -0.5, 0., 0.5, 1.0]
rareas = np.array([np.pi*((10**redges[i+1])**2. - (10**redges[i])**2.) for i in np.arange(len(redges)-1)])

In [16]:
magcol = "mag_i"
ii = ((table[magcol] > mag_lims[0]) & (table[magcol] < mag_lims[1]))

clust_los_nums = np.histogram(np.log10(table[ii]["R"]), bins=redges)[0] / 41 # / nc

ii = ((refpixel[magcol] > mag_lims[0]) & (refpixel[magcol] < mag_lims[1]))
surfdens = len(refpixel[ii]) / hp.nside2pixarea(32, degrees=True) / 3600 / 3
rands_los_nums = surfdens * rareas
# rands_los_nums = np.histogram(np.log10(refpixel[ii]["R"]), bins=redges)[0] / rareas#* ratio# / nr
nratios = clust_los_nums / rands_los_nums

/home/moon/vargatn/anaconda3/envs/galsim/lib/python3.7/site-packages/pandas/core/arraylike.py:358: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [18]:
ifields2 = []
iclusts2 = []
i2ds2 = []
for rbin in np.arange(4):
    print(rbin)
    _ifield, _iclust, _i2d = reader.result_reader2(samples[rbin], scores[rbin], nratio=nratios[rbin], m_factor=100, seed=rbin)
    ifields2.append(_ifield)
    iclusts2.append(_iclust)
    i2ds2.append(_i2d)

0
2.741989144315195
1
1.7913041757345514
2
1.1243442810539395
3
0.793271619957704


In [19]:
csamples = []
for rbin in np.arange(4):
    print(rbin)
    tab = pd.DataFrame.from_records(samples[rbin][iclusts2[rbin]].byteswap().newbyteorder())
    tab.drop('index', axis=1, inplace=True)
    kde = emulator.KDEContainer(tab)
    kde.standardize_data()
    kde.construct_kde(0.1)
    _csample = kde.random_draw(4e6)
    csamples.append(_csample)


0
1
2
3


In [20]:
rsamples = []
for rbin in np.arange(4):
    print(rbin)
    tab = pd.DataFrame.from_records(samples[rbin][ifields2[rbin]].byteswap().newbyteorder())
    tab.drop('index', axis=1, inplace=True)
    kde = emulator.KDEContainer(tab)
    kde.standardize_data()
    kde.construct_kde(0.1)
    _rsample = kde.random_draw(4e6)
    rsamples.append(_rsample)

0
1
2
3


In [21]:
allsamples = []
for rbin in np.arange(4):
    print(rbin)
    tab = pd.DataFrame.from_records(samples[rbin][i2ds2[rbin]].byteswap().newbyteorder())
    tab.drop('index', axis=1, inplace=True)
    kde = emulator.KDEContainer(tab)
    kde.standardize_data()
    kde.construct_kde(0.1)
    _sample = kde.random_draw(4e6)
    allsamples.append(_sample)

0
1
2
3


# Constructing a synthetic galaxy cluster

This is composed of two parts

1) Creating the bright central galaxy. For now this is set Manually from Varga et al, as it must be fitted to the outer stellar envelope model introduced in notebook C3

2) Drawing a cluster member galaxy population from the KDE model produced in notebooks A1 and A2

## The bright central galaxy (BCG) component set by hand

In [22]:
bcgtab = pd.DataFrame()
bcgtab["RA"] = [0,]
bcgtab["DEC"] = [0.,]
bcgtab["X"] = [0.,]
bcgtab["Y"] = [0.,]

_imag = 17.62
_gr = 1.38
_ri = 0.54
_iz = 0.37
bcgtab["MAG_G"] = [_gr + _ri + _imag,]
bcgtab["MAG_R"] = [_ri + _imag,]
bcgtab["MAG_I"] = [_imag,]
bcgtab["MAG_Z"] = [_imag - _iz,]
bcgtab["FRACDEV"] = [1.,]
bcgtab["TSIZE"] = [33.20,]

bcg_gabs = 0.14
# bcg_gabs = 0.07
angle = np.random.uniform(0, np.pi, size=1)

bcgtab["G1"] = -1. * bcg_gabs * np.cos(2 * angle)
bcgtab["G2"] = -1. * bcg_gabs * np.sin(2 * angle)
# bcgtab["G1"] = [0,]
# bcgtab["G2"] = [0,]
bcgtab["LOC"] = 1

bcgtab["Z"] = 0.325
bcgtab["shear"] = 0.
# bcgtab["shear2"] = 0.
# bcgtab["Z"]

bcgtab["FLUX_G"] = 10 ** (0.4 *(30. - bcgtab["MAG_G"]))
bcgtab["FLUX_R"] = 10 ** (0.4 *(30. - bcgtab["MAG_R"]))
bcgtab["FLUX_I"] = 10 ** (0.4 *(30. - bcgtab["MAG_I"]))
bcgtab["FLUX_Z"] = 10 ** (0.4 *(30. - bcgtab["MAG_Z"]))

## Drawing the cluster member population

In [23]:
ctab = _wide_cr_settings_clust["container"].data
rtab = _wide_cr_settings_rands["container"].data

In [24]:
magcol = "mag_i"
magcol_sample = "MAG_I"
rcol = "LOGR"
mag_lims = (17, 22.5)
medges = np.linspace(17, 22.5, 6)
medges2 = np.linspace(22.5, 24.5, 4)

mag_slices = [(17, 19), (19, 20), (20, 21), (21, 22), (22, 22.5)]

r_lims_all = [(-1.5, -0.5), (-0.5, 0.), (0, 0.5), (0.5, 1.0)]
redges = [-1.5, -0.5, 0., 0.5, 1.0]
rareas = np.array([np.pi*((10**redges[i+1])**2. - (10**redges[i])**2.) for i in np.arange(len(redges)-1)])


ii = ((table[magcol] > mag_lims[0]) & (table[magcol] < mag_lims[1]))

clust_los_nums = np.histogram(np.log10(table[ii]["R"]), bins=redges)[0] / 41 # / nc

ii = ((refpixel[magcol] > mag_lims[0]) & (refpixel[magcol] < mag_lims[1]))
surfdens = len(refpixel[ii]) / hp.nside2pixarea(32, degrees=True) / 3600 / 3
rands_los_nums = surfdens * rareas
# rands_los_nums = np.histogram(np.log10(refpixel[ii]["R"]), bins=redges)[0] / rareas#* ratio# / nr
nratios = np.max((clust_los_nums / rands_los_nums, np.ones(4)), axis=0)
# TODO there is a footprint masking bug which is fixed in the above line

frac = 1. / (clust_los_nums / rands_los_nums)
fac = 1
clust_nums_to_draw = clust_los_nums * (1 - frac) * fac
rands_nums_to_draw = rands_los_nums * fac


maglim_max1 = 22.5
maglim_max2 = 24.5
clust_gals = []
field_gals = []

# Becouse of the above surface density bug, we restrict the radial range to the inner 3 radial ranges
for rbin, r_lims in enumerate(r_lims_all[:3]):
    print(rbin, r_lims)

    #######################################
    sample = csamples[rbin]

    ii = ((np.log10(table["R"]) > r_lims[0]) & (np.log10(table["R"]) < r_lims[1]))
    vals = np.histogram(table[ii][magcol], bins=medges)[0]
    vals = vals / vals.sum()

    for i in np.arange(len(medges) - 1):
        _n_to_draw = clust_nums_to_draw[rbin] * vals[i]
        n_to_draw = np.random.poisson(_n_to_draw)

        index = ((sample[magcol_sample] > medges[i]) & (sample[magcol_sample] < medges[i + 1]) &
                 (sample[rcol] > r_lims[0]) & (sample[rcol] < r_lims[1])).values.astype(bool)
        ii = np.random.randint(0, len(sample.iloc[index]), size=n_to_draw)
        tmp = sample.iloc[index].iloc[ii]
        if len(tmp):
            clust_gals.append(tmp)   

    ii = ((sample[magcol_sample] > mag_lims[0]) & (sample[magcol_sample] < maglim_max1) &
          (sample[rcol] > r_lims[0]) & (sample[rcol] < r_lims[1]))
    num1 = len(sample[ii])
# 
    ii = ((sample[magcol_sample] > maglim_max1) & (sample[magcol_sample] < maglim_max2) &
          (sample[rcol] > r_lims[0]) & (sample[rcol] < r_lims[1]))
    num2 = len(sample[ii])
    fac_clust = num2 / num1

    vals = np.histogram(sample[magcol_sample], bins=medges2)[0]
    vals = vals / vals.sum()
    for i in np.arange(len(medges2) - 1):
        _n_to_draw = clust_nums_to_draw[rbin] * vals[i] * fac_clust
        n_to_draw = np.random.poisson(_n_to_draw)

        index = ((sample[magcol_sample] > medges2[i]) & (sample[magcol_sample] < medges2[i + 1]) &
        (sample[rcol] > r_lims[0]) & (sample[rcol] < r_lims[1])).values.astype(bool)
#         ii = np.random.randint(0, len(sample.iloc[index]), size=n_to_draw)
#         tmp = sample.iloc[index].iloc[ii]
        if len(tmp):
            clust_gals.append(tmp)   



gals = pd.concat(clust_gals, sort=True)
gals["Z"] = 0.325


/home/moon/vargatn/anaconda3/envs/galsim/lib/python3.7/site-packages/pandas/core/arraylike.py:358: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)


0 (-1.5, -0.5)


/home/moon/vargatn/anaconda3/envs/galsim/lib/python3.7/site-packages/pandas/core/arraylike.py:358: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)
/home/moon/vargatn/anaconda3/envs/galsim/lib/python3.7/site-packages/pandas/core/arraylike.py:358: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)


1 (-0.5, 0.0)


/home/moon/vargatn/anaconda3/envs/galsim/lib/python3.7/site-packages/pandas/core/arraylike.py:358: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)
/home/moon/vargatn/anaconda3/envs/galsim/lib/python3.7/site-packages/pandas/core/arraylike.py:358: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)


2 (0, 0.5)


/home/moon/vargatn/anaconda3/envs/galsim/lib/python3.7/site-packages/pandas/core/arraylike.py:358: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)
/home/moon/vargatn/anaconda3/envs/galsim/lib/python3.7/site-packages/pandas/core/arraylike.py:358: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [25]:
gals

,COLOR_G_R,COLOR_I_Z,COLOR_R_I,GABS,HALO_MASS,LOGR,MAG_I,SIZE,STELLAR_MASS,Z
1619489,1.384964,0.233583,0.585882,0.036762,12.022734,-0.796144,20.138684,-0.327499,9.450270,0.325
1192964,1.492789,0.307177,0.569517,0.020267,11.651747,-1.042950,21.471491,-0.309659,9.788993,0.325
1192964,1.492789,0.307177,0.569517,0.020267,11.651747,-1.042950,21.471491,-0.309659,9.788993,0.325
1192964,1.492789,0.307177,0.569517,0.020267,11.651747,-1.042950,21.471491,-0.309659,9.788993,0.325
1192964,1.492789,0.307177,0.569517,0.020267,11.651747,-1.042950,21.471491,-0.309659,9.788993,0.325
...,...,...,...,...,...,...,...,...,...,...
609687,1.400054,0.332818,0.597091,0.063881,11.589707,0.118166,21.541337,-0.157584,9.819496,0.325
338557,0.751945,0.167074,0.267254,0.082572,11.038210,0.458294,21.659119,-0.608504,8.313305,0.325
3343833,1.576798,0.356889,1.130035,0.106113,13.801876,0.286246,22.010317,-0.488700,10.069775,0.325
575315,1.000157,0.148726,0.360179,0.278130,10.698499,0.405593,22.191728,-0.816699,8.511283,0.325
